In [18]:
from dotenv import load_dotenv
import os
%load_ext autoreload
%autoreload 2

load_dotenv()  # loads .env from current directory

True

In [ ]:
from datasets import load_dataset

ds = load_dataset("csv", data_files="sample.csv")
ds = ds["train"]

Generating train split: 100 examples [00:00, 4339.37 examples/s]


In [58]:
from groq_helper import generate_summary_groq, get_groq_client

key = os.getenv("groq")
client = get_groq_client(key)

summary = generate_summary_groq(client=client, word_count=100, model="gemma2-9b-it", original_text=ds[0]["text"])

generated sequence of length =  195


In [60]:
def create_summary(example, word_count):
    column = f"{word_count}_words_gemma"
    if column not in example.keys() or len(example[column]) < 1:
        summary = generate_summary_groq(client=client, model="gemma2-9b-it", word_count=word_count, original_text=example["text"])
        example[f"{word_count}_words_gemma"] = summary
    return example

In [69]:
ds = ds.map(lambda ex: create_summary(example=ex, word_count=5))

Map: 100%|██████████| 100/100 [00:00<00:00, 268.02 examples/s]

error! returning empty


In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.getenv("openai")
)

def create_chatgpt_summary(word_count, instruction, example, model="gpt-4o-mini"):
  article = example["text"]
  completion = client.responses.create(
    model=model,
    store=True,
    input=f"Generate an exactly {word_count} word summary of the following article. {instruction}. \n {article}"
  )
  message = completion.output[0].content[0].text
  print("generated sequence of length = ", len("".join(message).split()))
  #print("message = ", message)

  return message
  #example[f"{word_count}_words_gpt"] = message.content
  #return example



In [ ]:
ds = ds.map(lambda ex: create_chatgpt_summary(example=ex, word_count=100))

In [73]:
import pandas as pd

df = ds.to_pandas()

In [74]:
df.columns

Index(['topic', 'date', 'tags', 'bias', 'text', 'link', 'source', 'title',
       'id', 'int_bias', '100_words_llama', '5_words_llama', '5_words_gpt',
       '100_words_gpt', '5_words_gemma', '100_words_gemma'],
      dtype='object')

In [76]:
print(f"column=100_words_llama, average={df['100_words_llama'].apply(lambda x: len(x.split())).mean()}")
print(f"column=5_words_llama, average={df['5_words_llama'].apply(lambda x: len(x.split())).mean()}")
print(f"column=100_words_gpt, average={df['100_words_gpt'].apply(lambda x: len(x.split())).mean()}")
print(f"column=5_words_gpt, average={df['5_words_gpt'].apply(lambda x: len(x.split())).mean()}")
print(f"column=100_words_gemma, average={df['100_words_gemma'].apply(lambda x: len(x.split())).mean()}")
print(f"column=5_words_gemma, average={df['5_words_gemma'].apply(lambda x: len(x.split())).mean()}")

column=100_words_llama, average=69.54
column=5_words_llama, average=4.33
column=100_words_gpt, average=98.75
column=5_words_gpt, average=5.72
column=100_words_gemma, average=119.19
column=5_words_gemma, average=16.99


In [78]:
ds.select_columns(column_names=['id','text','100_words_llama', '5_words_llama', '5_words_gpt','100_words_gpt']).to_csv("gpt_vs_llama.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 71.78ba/s]


707087

In [85]:
ds[7]

{'topic': 'white_house',
 'date': '2017-08-18',
 'tags': None,
 'bias': 'right',
 'text': 'Former White House Chief Strategist Steve Bannon ’ s government identification card ( Reuters : Carlos Barria )\nIdeology is just the furniture and props that the characters throw at each other in set-piece fights .\nThere ’ s a mismatch between the political press and the White House it is intended to cover . Political scribblers like myself are primed to greet the news of Steve Bannon getting bounced from the White House as an event with significance for the battle of ideas in the White House and in the Republican party .\nWe want to say that Bannon ’ s a “ populist ” of some sort , an anti-China guy , an America Firster . Heck , he even advocated tax raises on high-income earners . And we want to say that his enemies in the White House are either social-climbing socialites ( liberals ) , generals who favor more military engagement ( hawks ) , or business guys who think government is here to go

In [93]:
create_chatgpt_summary(10, ds[7])

generated sequence of length =  11
message =  Steve Bannon's departure fuels political theater, leaving press bewildered and entertained.


{'topic': 'white_house',
 'date': '2017-08-18',
 'tags': None,
 'bias': 'right',
 'text': 'Former White House Chief Strategist Steve Bannon ’ s government identification card ( Reuters : Carlos Barria )\nIdeology is just the furniture and props that the characters throw at each other in set-piece fights .\nThere ’ s a mismatch between the political press and the White House it is intended to cover . Political scribblers like myself are primed to greet the news of Steve Bannon getting bounced from the White House as an event with significance for the battle of ideas in the White House and in the Republican party .\nWe want to say that Bannon ’ s a “ populist ” of some sort , an anti-China guy , an America Firster . Heck , he even advocated tax raises on high-income earners . And we want to say that his enemies in the White House are either social-climbing socialites ( liberals ) , generals who favor more military engagement ( hawks ) , or business guys who think government is here to go

In [101]:
from IPython.display import display, HTML

def wrap_output(text):
    display(HTML(f"<div style='white-space: pre-wrap; word-wrap: break-word;'>{text}</div>"))


In [117]:
def create_chatgpt_summary(word_count, instruction, example, model="gpt-4o-mini"):
  article = example["text"]
  completion = client.responses.create(
    model=model,
    store=True,
    input=f"Generate an exactly {word_count} word summary of the following article. {instruction}. \n {article}",
    text={
      "format" : {
        "name": "response_type",
        "schema": {
          "$schema": "https://json-schema.org/draft/2020-12/schema",
          "type": "object",
          "properties": {
            "article": {
              "type": "string"
            }
          },
          "required": ["article"],
          "additionalProperties": False
        },
        "type": "json_schema" 
        }
      }
  )
  message = completion.output_text
  print("generated sequence of length = ", len("".join(message).split()))
  #print("message = ", message)

  return message

In [137]:
def create_chatgpt_tags(word_count, instruction, example):
    article = example["text"]
    response = client.responses.create(model="gpt-4o-mini",
        store=True,
        input=f"Generate a list of at most {word_count} topics for the following article. {instruction}. \n {article}",
        text= {
        "format" : {
            "name": "response_type",
            "schema": {
                "$schema": "https://json-schema.org/draft/2020-12/schema",
                "type": "object",
                "properties": {
                    "response": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                    }
                },
                "required": ["response"],
                "additionalProperties": False
            },
            "type": "json_schema" 
            }
        },
    )
    return response.output_text

In [139]:
wrap_output(
    create_chatgpt_tags(
        word_count=5,
        instruction="Each topic must be 1-2 words long. Select topics which are the most important for the text",
        example=ds[7]
    )
)

In [132]:
wrap_output(create_chatgpt_summary(
    word_count=100, 
    instruction="Keep the authorial voice, perspective, and tone",#"Keep the same authorial voice as the original article.", 
    example=ds[7])
)

generated sequence of length =  102
